### Inicialización

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install_from_url("https://github.com/conda-forge/miniforge/releases/download/25.3.1-0/Miniforge3-Linux-x86_64.sh")

✨🍰✨ Everything looks OK!


In [ ]:
!conda install -q pyscipopt

Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pyscipopt


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ampl-asl-1.0.0             |       h5888daf_2         504 KB  conda-forge
    ca-certificates-2025.11.12 |       hbd8a1cb_0         149 KB  conda-forge
    certifi-2025.11.12         |     pyhd8ed1ab_0         153 KB  conda-forge
    conda-25.7.0               |  py312h7900ff3_0         1.2 MB  conda-forge
    cppad-20250000.2           |       h5888daf_0         487 KB  conda-forge
    gmp-6.3.0                  |       hac33072_2         449 KB  conda-forge
    ipopt-3.14.19              |       h0804adb_0        1000 KB  conda-forge
    libblas-3.11.0             |5_h4a7cf45_openblas          18 KB  conda-forge
    libcblas-3.11.0            |5_h0358290_ope

In [ ]:
# Importamos la clase Model de pyscipopt
from pyscipopt import Model, quicksum

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import time

import json


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 # Datos del problema
I = list(range(1, 11))   # 1: Argentina, 2: Brasil, 3: Uruguay, ...
K = list(range(1, 19))   # 18 fechas simples
K_odd = [1,3,5,7,9,11,13,15,17]  # cada fecha impar indica el comienzo de una fecha doble
I_s = [1,2]              # lista de top teams
n = len(I)               # número de equipos

In [ ]:
def crear_modelo(evitar_argentina_brasil=True):
    """
    Crea el modelo del problema.
    evitar_argentina_brasil=False quita la restricción de los partidos contra los top teams.
    """

    # Crear modelo
    model = Model("EliminatoriasSudamericanas")

    # Definición de las variables
    # x[i][j][k] = 1 si equipo i juega de local contra j en fecha k
    vars_x = {}
    for i in I:
        for j in I:
            if i != j:
                for k in K:
                    vars_x[(i, j, k)] = model.addVar(vtype="B", name=f"x_{i}_{j}_{k}")

    vars_y = {}
    # y[i][k] = 1 si equipo i tiene una secuencia local-visitante en la Fecha Fifa que comienza en k
    #las definimos solo para los k en K_odd
    for i in I:
        for k in K_odd:
            vars_y[(i, k)] = model.addVar(vtype="B", name=f"y_{i}_{k}")

    vars_w = {}
    # w[i][k] = 1 si equipo i tiene un double break de visitante en la Fecha FIFA que comienza en k
    #las definimos solo para los k en K_odd
    for i in I:
        for k in K_odd:
            vars_w[(i, k)] = model.addVar(vtype="B", name=f"w_{i}_{k}")

    # Restricciones del torneo doble round-robin
    # Primera mitad
    for i in I:
        for j in I:
            if i != j:
                model.addCons(
                    quicksum(vars_x[i, j, k] + vars_x[j, i, k] for k in range(1, n)) == 1
                )

    # Segunda mitad
    for i in I:
        for j in I:
            if i < j:
                model.addCons(
                    quicksum(vars_x[i, j, k] + vars_x[j, i, k] for k in range(n, 2*n-1)) == 1
                )

    # Cada par de partidos entre i y j se juega exactamente una vez en cada sede
    for i in I:
        for j in I:
            if i != j:
                model.addCons(
                    quicksum(vars_x[i, j, k] for k in K) == 1
                )

    # Compacidad: cada equipo juega exactamente un partido por fecha
    for j in I:
        for k in K:
            model.addCons(
                quicksum(vars_x[i, j, k] + vars_x[j, i, k] for i in I if i != j) == 1
            )

    # Restricciones para los equipos principales
    if evitar_argentina_brasil:
        for i in I:
            if i not in I_s:
                for k in range(1, len(K)):
                    model.addCons(
                        quicksum(vars_x[i, j, k] + vars_x[j, i, k] +
                                 vars_x[i, j, k+1] + vars_x[j, i, k+1] for j in I_s) <= 1
                    )

    # Balance
    for i in I:
        model.addCons(quicksum(vars_y[i, k] for k in K_odd) <= n/2)
        model.addCons(quicksum(vars_y[i, k] for k in K_odd) >= (n/2)-1)

    for i in I:
        for k in K_odd:
            model.addCons(
                quicksum(vars_x[i, j, k] + vars_x[j, i, k+1] for j in I if i != j) <= 1 + vars_y[i, k]
            )
            model.addCons(
                quicksum(vars_x[i, j, k] for j in I if i != j) >= vars_y[i, k]
            )
            model.addCons(
                quicksum(vars_x[j, i, k+1] for j in I if i != j) >= vars_y[i, k]
            )

    # Break visitante-visitante
    for i in I:
        for k in K_odd:
            model.addCons(
                quicksum(vars_x[j, i, k] + vars_x[j, i, k+1] for j in I if i != j) <= vars_w[i, k] + 1
            )
            model.addCons(
                quicksum(vars_x[j, i, k] for j in I if i != j) >= vars_w[i, k]
            )
            model.addCons(
                quicksum(vars_x[j, i, k+1] for j in I if i != j) >= vars_w[i, k]
            )

    # Función objetivo
    model.setObjective(
        quicksum(vars_w[i, k] for i in I for k in K_odd),
        "minimize"
    )

    return model, vars_x, vars_y, vars_w


In [ ]:
def resolver_y_mostrar(model, vars_x, vars_y, vars_w, titulo):
    """
    Resuelve el modelo y muestra los resultados mas relevantes.
    """
    model.optimize()

    # Verificamos si encontró solución
    if model.getObjVal() is not None:
        obj_val = int(model.getObjVal())
        print(f"Solución encontrada.")
        print(f"Valor Objetivo (Total Double Breaks): {obj_val}\n")

    # Mostrar detalle de Double Breaks (si hay)
        if obj_val > 0:
          print("--- Detalle de Double Breaks ---")
          hay_breaks = False
          # recorremos equipos y fechas FIFA impares (K_odd)
          for (i, k), var in vars_w.items():
            if model.getVal(var) > 0.5:
              print(f"  Equipo {i} tiene un DOUBLE BREAK de visitante en Fecha FIFA que comienza en {k}")
              hay_breaks = True
          if not hay_breaks:
            print("  (Error numérico o breaks ocultos)")

    else:
        print(" No se encontró solución óptima.")




In [ ]:
# PRUEBA BASE: Con y Sin Restricción Arg-Bra
#modelo sin restricciones de simetria

if __name__ == "__main__":

    # CON Restricción
    print("=== Modelo Base CON restricción Arg-Bra ===")
    m_con, x_con, y_con, w_con = crear_modelo(evitar_argentina_brasil=True)
    resolver_y_mostrar(m_con, x_con, y_con, w_con, "Base CON Restricción")

    print("\n" + "="*60 + "\n")

    # SIN Restricción
    print("=== Modelo Base SIN restricción Arg-Bra ===")
    m_sin, x_sin, y_sin, w_sin = crear_modelo(evitar_argentina_brasil=False)
    resolver_y_mostrar(m_sin, x_sin, y_sin, w_sin, "Base SIN Restricción")

=== Modelo Base CON restricción Arg-Bra ===
Solución encontrada.
Valor Objetivo (Total Double Breaks): 0



=== Modelo Base SIN restricción Arg-Bra ===
Solución encontrada.
Valor Objetivo (Total Double Breaks): 0



In [ ]:
#en ambos casos encontramos soluciones optimas
#a continuación, imprimimos el fixture en pantalla

In [ ]:
def mostrar_fixture(model, vars_x, I, K):
    """
    Muestra el fixture completo en formato tabla.
    """
    print("\n=== FIXTURE COMPLETO ===\n")
    for k in K:
        print(f"Fecha {k}:")
        partidos = []
        for i in I:
            for j in I:
                if i != j and model.getVal(vars_x[i, j, k]) > 0.5:
                    partidos.append((i, j))
        # Mostrar todos los partidos de la fecha
        for (i, j) in partidos:
            print(f"  Equipo {i} (local) vs Equipo {j} (visitante)")
        print()


In [ ]:
mostrar_fixture(m_con, x_con, I, K)
#fixture consisderando la restricción de top teams


=== FIXTURE COMPLETO ===

Fecha 1:
  Equipo 3 (local) vs Equipo 2 (visitante)
  Equipo 4 (local) vs Equipo 9 (visitante)
  Equipo 5 (local) vs Equipo 7 (visitante)
  Equipo 8 (local) vs Equipo 6 (visitante)
  Equipo 10 (local) vs Equipo 1 (visitante)

Fecha 2:
  Equipo 1 (local) vs Equipo 5 (visitante)
  Equipo 2 (local) vs Equipo 4 (visitante)
  Equipo 6 (local) vs Equipo 3 (visitante)
  Equipo 7 (local) vs Equipo 8 (visitante)
  Equipo 9 (local) vs Equipo 10 (visitante)

Fecha 3:
  Equipo 1 (local) vs Equipo 3 (visitante)
  Equipo 2 (local) vs Equipo 8 (visitante)
  Equipo 4 (local) vs Equipo 6 (visitante)
  Equipo 5 (local) vs Equipo 10 (visitante)
  Equipo 9 (local) vs Equipo 7 (visitante)

Fecha 4:
  Equipo 3 (local) vs Equipo 5 (visitante)
  Equipo 6 (local) vs Equipo 1 (visitante)
  Equipo 7 (local) vs Equipo 2 (visitante)
  Equipo 8 (local) vs Equipo 9 (visitante)
  Equipo 10 (local) vs Equipo 4 (visitante)

Fecha 5:
  Equipo 4 (local) vs Equipo 3 (visitante)
  Equipo 6 (local

In [ ]:
mostrar_fixture(m_sin, x_sin, I, K)
#fixture sin considerar la restriccion top teams


=== FIXTURE COMPLETO ===

Fecha 1:
  Equipo 3 (local) vs Equipo 2 (visitante)
  Equipo 5 (local) vs Equipo 7 (visitante)
  Equipo 6 (local) vs Equipo 1 (visitante)
  Equipo 8 (local) vs Equipo 4 (visitante)
  Equipo 10 (local) vs Equipo 9 (visitante)

Fecha 2:
  Equipo 1 (local) vs Equipo 3 (visitante)
  Equipo 2 (local) vs Equipo 10 (visitante)
  Equipo 4 (local) vs Equipo 5 (visitante)
  Equipo 7 (local) vs Equipo 8 (visitante)
  Equipo 9 (local) vs Equipo 6 (visitante)

Fecha 3:
  Equipo 1 (local) vs Equipo 4 (visitante)
  Equipo 3 (local) vs Equipo 7 (visitante)
  Equipo 6 (local) vs Equipo 2 (visitante)
  Equipo 8 (local) vs Equipo 9 (visitante)
  Equipo 10 (local) vs Equipo 5 (visitante)

Fecha 4:
  Equipo 2 (local) vs Equipo 8 (visitante)
  Equipo 4 (local) vs Equipo 10 (visitante)
  Equipo 5 (local) vs Equipo 6 (visitante)
  Equipo 7 (local) vs Equipo 1 (visitante)
  Equipo 9 (local) vs Equipo 3 (visitante)

Fecha 5:
  Equipo 2 (local) vs Equipo 1 (visitante)
  Equipo 4 (local

In [ ]:
#Esquemas
#a las restricciones anteriores agregamos restricciones de simetria
def aplicar_esquema_espejo(model, vars_x):
    """
    Agrega la restricción del esquema espejo.
    Paper Eq (14)
    """
    print(" -> Aplicando: ESQUEMA ESPEJO ")
    for k in range(1, n):
        for i in range(1, n+1):
            for j in range(1, n+1):
                if i != j: model.addCons(vars_x[(i, j, k)] == vars_x[(j, i, k + n-1)])

def aplicar_esquema_frances(model, vars_x):
    """
    Agrega la restricción del esquema francés.
    Paper Eq (15)
    """
    print(" -> Aplicando: ESQUEMA FRANCÉS")
    for i in range(1, n+1):
        for j in range(1, n+1):
            if i != j:
                model.addCons(vars_x[(i, j, 1)] == vars_x[(j, i, 2*n-2)])
                for k in range(2, 10):
                    model.addCons(vars_x[(i, j, k)] == vars_x[(j, i, k + n-2)])

def aplicar_esquema_ingles(model, vars_x):
    """
    Agrega la restricción del esquema inglés.
    Paper Eq (16)
    """
    print(" -> Aplicando: ESQUEMA INGLÉS ")
    for i in range(1, n+1):
        for j in range(1, n+1):
            if i != j:
                model.addCons(vars_x[(i, j, n-1)] == vars_x[(j, i, n)])
                for k in range(2, 9):
                    model.addCons(vars_x[(i, j, k)] == vars_x[(j, i, k + n)])

def aplicar_esquema_invertido(model, vars_x):
    """
    Agrega la restricción del esquema invertido.
    Paper Eq (17)
    """
    print(" -> Aplicando: ESQUEMA INVERTIDO ")
    for k in range(1, n):
        k_inv = 2*n -1 - k
        for i in range(1, n+1):
            for j in range(1, n+1):
                if i!=j: model.addCons(vars_x[(i, j, k)] == vars_x[(j, i, k_inv)])

def aplicar_esquema_minmax(model, vars_x):
    """
    Agrega la restricción del esquema min-max.
    Paper Eq (19) y Eq (20)
    """
    print(" -> Aplicando: MIN-MAX ")
    c=5
    d=13
    # Contadores para debugging
    restricciones_min = 0
    restricciones_max = 0

    # --- Restricción mínima (Eq. 19) ---
    # "No pueden jugar ida y vuelta en menos de c+1 fechas"
    # Suma desde k hasta k+c (es decir, c+1 fechas consecutivas)
    for i in range(1, n+1):
        for j in range(i+1, n+1):  # i<j para evitar duplicados
            # k puede ir desde 1 hasta fechas_totales-c
            for k in range(1, len(K) - c + 1):
                ventana = []
                for t in range(k, min(k + c + 1, len(K) + 1)):
                    ventana.append(vars_x[(i, j, t)])
                    ventana.append(vars_x[(j, i, t)])

                # A lo sumo 1 partido entre i y j en esta ventana
                model.addCons(quicksum(ventana) <= 1)
                restricciones_min += 1

    # --- Restricción máxima (Eq. 20) ---
    # "Si j juega de local contra i en k, la revancha debe estar a ≤d fechas"
    for i in range(1, n+1):
        for j in range(1, n+1):
            if i != j:
                for k in range(1, len(K) + 1):
                    # Ventana: [k-d, k+d] excluyendo k
                    posibles_revancha = []

                    inicio = max(1, k - d)
                    fin = min(len(K), k + d)

                    for t in range(inicio, fin + 1):
                        if t != k:
                            posibles_revancha.append(vars_x[(i, j, t)])

                    # Si x(j,i,k) = 1, entonces debe haber una revancha x(i,j,t)
                    # en algún t dentro de [k-d, k+d] \ {k}
                    if posibles_revancha:  # Solo si hay fechas posibles
                        model.addCons(quicksum(posibles_revancha) >= vars_x[(j, i, k)])
                        restricciones_max += 1

    print(f"    ✓ {restricciones_min} restricciones de separación mínima")
    print(f"    ✓ {restricciones_max} restricciones de separación máxima")
    print(f"    Total: {restricciones_min + restricciones_max} restricciones")


def aplicar_esquema_backtoback(model, vars_x):
    """
    Esquema BACK-TO-BACK (Ida y Vuelta inmediato):
    Los equipos juegan contra el mismo rival en la fecha k y k+1.
    Matemáticamente: x[i,j,k] == x[j,i,k+1] para fechas impares.
    """
    print(" -> Aplicando restricciones: ESQUEMA BACK-TO-BACK")
    for k in K_odd:
        if k+1 <= 2*(n-1):  # asegurar que existe la fecha siguiente
            for i in range(1, n+1):
                for j in range(1, n+1):
                    if i != j:
                        model.addCons(vars_x[(i, j, k)] == vars_x[(j, i, k+1)])


In [ ]:
#aplicamos esquemas y graficamos en caso de ser posible

In [ ]:
def ejecutar_esquema(nombre_esquema, usar_arg_bra=True):
    print(f"\n{'='*60}")
    print(f"PROCESANDO: {nombre_esquema} (Restricción Arg-Bra: {usar_arg_bra})")

    # Crear modelo base
    model, vars_x, vars_l, vars_v = crear_modelo(evitar_argentina_brasil=usar_arg_bra)

    model.setParam("limits/time", 18000) #5 hs de tiempo limite para encontrar soluciones

    # Aplicar esquema
    if nombre_esquema == "Frances":
        aplicar_esquema_frances(model, vars_x)
    elif nombre_esquema == "Espejo":
        aplicar_esquema_espejo(model, vars_x)
    elif nombre_esquema == "Ingles":
        aplicar_esquema_ingles(model, vars_x)
    elif nombre_esquema == "Invertido":
        aplicar_esquema_invertido(model, vars_x)
    elif nombre_esquema == "BackToBack":
        aplicar_esquema_backtoback(model, vars_x)
    elif nombre_esquema == "MinMax":
        aplicar_esquema_minmax(model, vars_x)

    # Optimizar
    start_time = time.time()
    model.optimize()
    end_time = time.time()
    elapsed = end_time - start_time
    print(f"Tiempo de resolución: {elapsed:.2f} segundos")

    # Estado del solver
    status = model.getStatus()
    print(f"Estado del Solver: {status}")

    if status == "optimal" or (status == "timelimit" and model.getSols()):
        breaks = int(model.getObjVal())
        num_sols = model.getNSols()
        print(f"Solución Encontrada.")
        print(f"Total Double Breaks: {breaks}")
        print(f"Cantidad de soluciones encontradas: {num_sols}")

        # Reconstruir fixture y retornarlo
        fixture_dict = {}
        for (i, j, k), var in vars_x.items():
            if model.getVal(var) > 0.5:
                fixture_dict.setdefault(k, []).append((i, j))

        return fixture_dict
    else:
        print("No se encontró solución factible (Infactible o Error).")
        return None

In [ ]:
# ZONA DE EJECUCIÓN
#Espejo con y sin arg_bra
fixture_espejo_con=ejecutar_esquema("Espejo", usar_arg_bra=True)
fixture_espejo_sin=ejecutar_esquema("Espejo", usar_arg_bra=False)

#Despues de correr nos dan los dos infactibles


PROCESANDO: Espejo (Restricción Arg-Bra: True)
 -> Aplicando: ESQUEMA ESPEJO 
Tiempo de resolución: 69.13 segundos
Estado del Solver: infeasible
No se encontró solución factible (Infactible o Error).

PROCESANDO: Espejo (Restricción Arg-Bra: False)
 -> Aplicando: ESQUEMA ESPEJO 
Tiempo de resolución: 41.06 segundos
Estado del Solver: infeasible
No se encontró solución factible (Infactible o Error).


In [ ]:
#Frances con y sin arg_bra
fixture_frances_con=ejecutar_esquema("Frances", usar_arg_bra=True)
fixture_frances_sin=ejecutar_esquema("Frances", usar_arg_bra=False)

#En ambos casos encuentra la sol optima y son ambas con 0 breaks


PROCESANDO: Frances (Restricción Arg-Bra: True)
 -> Aplicando: ESQUEMA FRANCÉS
Tiempo de resolución: 18.59 segundos
Estado del Solver: optimal
Solución Encontrada.
Total Double Breaks: 0
Cantidad de soluciones encontradas: 2

PROCESANDO: Frances (Restricción Arg-Bra: False)
 -> Aplicando: ESQUEMA FRANCÉS
Tiempo de resolución: 9.67 segundos
Estado del Solver: optimal
Solución Encontrada.
Total Double Breaks: 0
Cantidad de soluciones encontradas: 2


In [ ]:
# Guardar cada fixture en Drive
with open("/content/drive/MyDrive/TP_FINAL_INVOP/fixture_frances_con.json", "w") as f:
    json.dump(fixture_frances_con, f)
with open("/content/drive/MyDrive/TP_FINAL_INVOP/fixture_frances_sin.json", "w") as f:
    json.dump(fixture_frances_sin, f)

In [ ]:
#Ingles con y sin arg_bra
fixture_ingles_con=ejecutar_esquema("Ingles", usar_arg_bra=True)
fixture_ingles_sin=ejecutar_esquema("Ingles", usar_arg_bra=False)

#En ambos casos encuentra la sol optima y son ambas con 0 breaks


PROCESANDO: Ingles (Restricción Arg-Bra: True)
 -> Aplicando: ESQUEMA INGLÉS 
Tiempo de resolución: 7.62 segundos
Estado del Solver: optimal
Solución Encontrada.
Total Double Breaks: 0
Cantidad de soluciones encontradas: 2

PROCESANDO: Ingles (Restricción Arg-Bra: False)
 -> Aplicando: ESQUEMA INGLÉS 
Tiempo de resolución: 7.88 segundos
Estado del Solver: optimal
Solución Encontrada.
Total Double Breaks: 0
Cantidad de soluciones encontradas: 2


In [ ]:
with open("/content/drive/MyDrive/TP_FINAL_INVOP/fixture_ingles_con.json", "w") as f:
    json.dump(fixture_ingles_con, f)
with open("/content/drive/MyDrive/TP_FINAL_INVOP/fixture_ingles_sin.json", "w") as f:
    json.dump(fixture_ingles_sin, f)

In [ ]:
#Invertido con y sin arg_bra
fixture_invertido_con=ejecutar_esquema("Invertido", usar_arg_bra=True)
fixture_invertido_sin=ejecutar_esquema("Invertido", usar_arg_bra=False)

#En ambos casos encuentra la sol optima y son ambas con 0 breaks


PROCESANDO: Invertido (Restricción Arg-Bra: True)
 -> Aplicando: ESQUEMA INVERTIDO 
Tiempo de resolución: 32.51 segundos
Estado del Solver: optimal
Solución Encontrada.
Total Double Breaks: 0
Cantidad de soluciones encontradas: 1

PROCESANDO: Invertido (Restricción Arg-Bra: False)
 -> Aplicando: ESQUEMA INVERTIDO 
Tiempo de resolución: 56.92 segundos
Estado del Solver: optimal
Solución Encontrada.
Total Double Breaks: 0
Cantidad de soluciones encontradas: 1


In [ ]:
with open("/content/drive/MyDrive/TP_FINAL_INVOP/fixture_invertido_con.json", "w") as f:
    json.dump(fixture_invertido_con, f)
with open("/content/drive/MyDrive/TP_FINAL_INVOP/fixture_invertido_sin.json", "w") as f:
    json.dump(fixture_invertido_sin, f)

In [ ]:
#BackToBack con y sin arg_bra
fixture_backtoback_con=ejecutar_esquema("BackToBack", usar_arg_bra=True)
fixture_backtoback_sin=ejecutar_esquema("BackToBack", usar_arg_bra=False)

#solucion infactible, es coherente con lo que dice el paper


PROCESANDO: BackToBack (Restricción Arg-Bra: True)
 -> Aplicando restricciones: ESQUEMA BACK-TO-BACK
Tiempo de resolución: 0.04 segundos
Estado del Solver: infeasible
No se encontró solución factible (Infactible o Error).

PROCESANDO: BackToBack (Restricción Arg-Bra: False)
 -> Aplicando restricciones: ESQUEMA BACK-TO-BACK
Tiempo de resolución: 0.03 segundos
Estado del Solver: infeasible
No se encontró solución factible (Infactible o Error).


In [ ]:
#Min-max con y sin arg_bra
fixture_minmax_con=ejecutar_esquema("MinMax", usar_arg_bra=True)
fixture_minmax_sin=ejecutar_esquema("MinMax", usar_arg_bra=False)



PROCESANDO: MinMax (Restricción Arg-Bra: True)
 -> Aplicando: MIN-MAX 
    ✓ 585 restricciones de separación mínima
    ✓ 1620 restricciones de separación máxima
    Total: 2205 restricciones
Tiempo de resolución: 8373.36 segundos
Estado del Solver: optimal
Solución Encontrada.
Total Double Breaks: 0
Cantidad de soluciones encontradas: 2

PROCESANDO: MinMax (Restricción Arg-Bra: False)
 -> Aplicando: MIN-MAX 
    ✓ 585 restricciones de separación mínima
    ✓ 1620 restricciones de separación máxima
    Total: 2205 restricciones
Tiempo de resolución: 607.12 segundos
Estado del Solver: userinterrupt
No se encontró solución factible (Infactible o Error).


In [ ]:
with open("/content/drive/MyDrive/TP_FINAL_INVOP/fixture_minmax_con.json", "w") as f:
    json.dump(fixture_minmax_con, f)

In [ ]:
fixture_minmax_sin=ejecutar_esquema("MinMax", usar_arg_bra=False)


PROCESANDO: MinMax (Restricción Arg-Bra: False)
 -> Aplicando: MIN-MAX 
    ✓ 585 restricciones de separación mínima
    ✓ 1620 restricciones de separación máxima
    Total: 2205 restricciones
Tiempo de resolución: 3173.99 segundos
Estado del Solver: optimal
Solución Encontrada.
Total Double Breaks: 0
Cantidad de soluciones encontradas: 1


In [ ]:
with open("/content/drive/MyDrive/TP_FINAL_INVOP/fixture_minmax_sin.json", "w") as f:
    json.dump(fixture_minmax_sin, f)

In [ ]:
#relajación de restricciones

In [ ]:
#ejecutamos el modelo de BackToBack relajando que se enfrenten todos con todos en las dos mitades del torneo.

In [ ]:
def crear_modelo_relajado(evitar_argentina_brasil=True):
    """
    Crea el modelo del problema.
    evitar_argentina_brasil=False quita la restricción de los partidos contra los top teams.
    """

    # Crear modelo
    model = Model("EliminatoriasSudamericanas")

    # Definición de las variables
    # x[i][j][k] = 1 si equipo i juega de local contra j en fecha k
    vars_x = {}
    for i in I:
        for j in I:
            if i != j:
                for k in K:
                    vars_x[(i, j, k)] = model.addVar(vtype="B", name=f"x_{i}_{j}_{k}")

    vars_y = {}
    # y[i][k] = 1 si equipo i tiene una secuencia local-visitante en la Fecha Fifa que comienza en k
    #las definimos solo para los k en K_odd
    for i in I:
        for k in K_odd:
            vars_y[(i, k)] = model.addVar(vtype="B", name=f"y_{i}_{k}")

    vars_w = {}
    # w[i][k] = 1 si equipo i tiene un double break de visitante en la Fecha FIFA que comienza en k
    #las definimos solo para los k en K_odd
    for i in I:
        for k in K_odd:
            vars_w[(i, k)] = model.addVar(vtype="B", name=f"w_{i}_{k}")

    # Restricciones del torneo doble round-robin

    # Cada par de partidos entre i y j se juega exactamente una vez en cada sede
    for i in I:
        for j in I:
            if i != j:
                model.addCons(
                    quicksum(vars_x[i, j, k] for k in K) == 1
                )

    # Compacidad: cada equipo juega exactamente un partido por fecha
    for j in I:
        for k in K:
            model.addCons(
                quicksum(vars_x[i, j, k] + vars_x[j, i, k] for i in I if i != j) == 1
            )

    # Restricciones para los equipos principales
    if evitar_argentina_brasil:
        for i in I:
            if i not in I_s:
                for k in range(1, len(K)):
                    model.addCons(
                        quicksum(vars_x[i, j, k] + vars_x[j, i, k] +
                                 vars_x[i, j, k+1] + vars_x[j, i, k+1] for j in I_s) <= 1
                    )

    # Balance
    for i in I:
        model.addCons(quicksum(vars_y[i, k] for k in K_odd) <= n/2)
        model.addCons(quicksum(vars_y[i, k] for k in K_odd) >= (n/2)-1)

    for i in I:
        for k in K_odd:
            model.addCons(
                quicksum(vars_x[i, j, k] + vars_x[j, i, k+1] for j in I if i != j) <= 1 + vars_y[i, k]
            )
            model.addCons(
                quicksum(vars_x[i, j, k] for j in I if i != j) >= vars_y[i, k]
            )
            model.addCons(
                quicksum(vars_x[j, i, k+1] for j in I if i != j) >= vars_y[i, k]
            )

    # Break visitante-visitante
    for i in I:
        for k in K_odd:
            model.addCons(
                quicksum(vars_x[j, i, k] + vars_x[j, i, k+1] for j in I if i != j) <= vars_w[i, k] + 1
            )
            model.addCons(
                quicksum(vars_x[j, i, k] for j in I if i != j) >= vars_w[i, k]
            )
            model.addCons(
                quicksum(vars_x[j, i, k+1] for j in I if i != j) >= vars_w[i, k]
            )

    # Función objetivo
    model.setObjective(
        quicksum(vars_w[i, k] for i in I for k in K_odd),
        "minimize"
    )

    return model, vars_x, vars_y, vars_w


In [ ]:
def ejecutar_esquema_relajado(usar_arg_bra=True):
    print(f"\n{'='*60}")
    print(f"PROCESANDO: BackToBack relajado (Restricción Arg-Bra: {usar_arg_bra})")

    # Crear modelo base
    model, vars_x, vars_l, vars_v = crear_modelo_relajado(evitar_argentina_brasil=usar_arg_bra)

    # Aplicar esquema
    aplicar_esquema_backtoback(model, vars_x)


    # Optimizar
    start_time = time.time()
    model.optimize()
    end_time = time.time()
    elapsed = end_time - start_time
    print(f"Tiempo de resolución: {elapsed:.2f} segundos")

    # Estado del solver
    status = model.getStatus()
    print(f"Estado del Solver: {status}")

    if status == "optimal" or (status == "timelimit" and model.getSols()):
        breaks = int(model.getObjVal())
        num_sols = model.getNSols()
        print(f"Solución Encontrada.")
        print(f"Total Double Breaks: {breaks}")
        print(f"Cantidad de soluciones encontradas: {num_sols}")

        # Reconstruir fixture y retornarlo
        fixture_dict = {}
        for (i, j, k), var in vars_x.items():
            if model.getVal(var) > 0.5:
                fixture_dict.setdefault(k, []).append((i, j))

        return fixture_dict
    else:
        print("No se encontró solución factible (Infactible o Error).")
        return None


In [ ]:
#BackToBack relajado con y sin arg_bra
fixture_backtoback_relajado_con=ejecutar_esquema_relajado(usar_arg_bra=True)
fixture_backtoback_relajado_sin=ejecutar_esquema_relajado(usar_arg_bra=False)


PROCESANDO: BackToBack relajado (Restricción Arg-Bra: True)
 -> Aplicando restricciones: ESQUEMA BACK-TO-BACK
Tiempo de resolución: 0.02 segundos
Estado del Solver: infeasible
No se encontró solución factible (Infactible o Error).

PROCESANDO: BackToBack relajado (Restricción Arg-Bra: False)
 -> Aplicando restricciones: ESQUEMA BACK-TO-BACK
Tiempo de resolución: 3.15 segundos
Estado del Solver: optimal
Solución Encontrada.
Total Double Breaks: 0
Cantidad de soluciones encontradas: 1


In [ ]:
with open("/content/drive/MyDrive/TP_FINAL_INVOP/fixture_backtoback_relajado_sin.json", "w") as f:
    json.dump(fixture_backtoback_relajado_sin, f)
